In [9]:
import pandas as pd
import osmnx as ox
import networkx as nx
import random
import math
import folium
import os
import time
import numpy as np

In [10]:
# --- KONFIGURASI ---
filename = '../Dataset/jne_solo.csv'    # Pastikan file ini ada
nama_folder_output = "../Result"        # Folder output
KAPASITAS_TRUK = 200                    # Kapasitas Truk Canter (Paket)
TARGET_TOTAL_PAKET = 400                # Total paket dummy

# KOORDINAT HARDCODE JNE WAREHOUSE ADI SUMARMO (COLOMADU)
WAREHOUSE_LAT = -7.532519372469423 
WAREHOUSE_LON = 110.76423789494254 

print("Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).")

Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).


In [11]:
# ==========================================
# LOAD DATA & GENERATE PAKET
# ==========================================
# 1. LOAD DATA AGEN
print("\n[Loading Data Agen...]")
try:
    df_jne = pd.read_csv(filename, sep=';')
    df_jne[['lat', 'lon']] = df_jne['Koordinat'].str.split(',', expand=True).astype(float)
    
    # Override lokasi Warehouse dengan input manual (Adi Sumarmo)
    warehouse_lat = WAREHOUSE_LAT
    warehouse_lon = WAREHOUSE_LON
    
    print(f"{len(df_jne)} Agen Loaded.")
    print(f"Warehouse: JNE Adi Sumarmo ({warehouse_lat}, {warehouse_lon})")

except Exception as e:
    print(f"Error: {e}")

# 2. GENERATE PAKET
print("\n[Generating Paket...]")
all_paket_data = []
paket_per_agen = TARGET_TOTAL_PAKET // len(df_jne) 
sisa_paket = TARGET_TOTAL_PAKET % len(df_jne)
counter_id = 1

def get_random_point_around(lat, lon, radius_km):
    r = radius_km / 111.0
    u = random.random()
    v = random.random()
    w = r * math.sqrt(u)
    t = 2 * math.pi * v
    x = w * math.cos(t)
    y_offset = w * math.sin(t)
    return lat + y_offset, lon + (x / math.cos(math.radians(lat)))

for idx, row in df_jne.iterrows():
    jatah = paket_per_agen + (1 if idx < sisa_paket else 0)
    for _ in range(jatah):
        new_lat, new_lon = get_random_point_around(row['lat'], row['lon'], 2.5)
        all_paket_data.append({
            'ID Paket': f'PKT-{counter_id:04d}',
            'lat': new_lat,
            'lon': new_lon,
            'Agen Induk': row['Nama JNE'],
            'Agen Lat': row['lat'],
            'Agen Lon': row['lon']
        })
        counter_id += 1

df_paket = pd.DataFrame(all_paket_data)
print(f"Cell 2: {len(df_paket)} Paket Generated.")


[Loading Data Agen...]
48 Agen Loaded.
Warehouse: JNE Adi Sumarmo (-7.532519372469423, 110.76423789494254)

[Generating Paket...]
Cell 2: 400 Paket Generated.


In [12]:
# ==========================================
# 3. DOWNLOAD PETA (FULL KARESIDENAN SURAKARTA)
# ==========================================
print("\n[Downloading Map: Karesidenan Surakarta...]")
wilayah_solo_raya = [
    "Surakarta, Central Java, Indonesia",
    "Sukoharjo, Central Java, Indonesia",
    "Karanganyar, Central Java, Indonesia", 
    "Boyolali, Central Java, Indonesia",
    "Klaten, Central Java, Indonesia",
    "Sragen, Central Java, Indonesia",
    "Wonogiri, Central Java, Indonesia"
]

try:
    # Download graph gabungan
    G = ox.graph_from_place(wilayah_solo_raya, network_type='drive')
    print(f"Cell 3: Peta Berhasil Didownload! ({len(G.nodes)} Simpul)")
except Exception as e:
    print(f"Gagal download peta: {e}")


[Downloading Map: Karesidenan Surakarta...]
Cell 3: Peta Berhasil Didownload! (322827 Simpul)


In [15]:
# ==========================================
# 4. HITUNG RUTE (REAL ROAD DISTANCE)
# ==========================================
print("\n[Cell 4: Menghitung Rute 1 Truk (Multi-Trip CVRP)...]")

# --- A. RUTE TRUK (CVRP: Farthest First - Single Vehicle) ---
print("Menghitung Trip Truk (Bolak-balik Warehouse)...")

node_warehouse = ox.nearest_nodes(G, WAREHOUSE_LON, WAREHOUSE_LAT)
demand_per_agen = df_paket['Agen Induk'].value_counts().to_dict()

# Data Agen yang harus dikunjungi
agen_to_visit = []
for idx, row in df_jne.iterrows():
    node = ox.nearest_nodes(G, row['lon'], row['lat'])
    demand = demand_per_agen.get(row['Nama JNE'], 0)
    if demand > 0:
        agen_to_visit.append({
            'Nama': row['Nama JNE'], 
            'Node': node, 
            'Demand': demand, 
            'lat': row['lat'], 'lon': row['lon']
        })

# Variabel Utama
truk_trips = []             # List menyimpan geometri jalan per trip
visited_agents_per_trip = [] # List menyimpan data agen per trip (buat marker)

# Status Awal
current_capacity = KAPASITAS_TRUK
current_node = node_warehouse
unvisited_agen = agen_to_visit.copy()
path_nodes_truk = [node_warehouse]

current_trip_agents = [] # Agen yg dikunjungi di trip SAAT INI
visit_order_total = 1    # Urutan kunjungan global

while unvisited_agen:
    target_agen = None
    idx_remove = -1
    is_at_warehouse = (current_node == node_warehouse)
    
    # Logika Optimasi:
    # - Di Gudang: Cari Agen TERJAUH (Farthest First)
    # - Di Jalan: Cari Agen TERDEKAT (Nearest Neighbor)
    best_metric = -1.0 if is_at_warehouse else float('inf')

    for i, agen in enumerate(unvisited_agen):
        # Cek Kapasitas
        if agen['Demand'] <= current_capacity:
            try:
                dist = nx.shortest_path_length(G, current_node, agen['Node'], weight='length')
                if is_at_warehouse:
                    if dist > best_metric: best_metric = dist; target_agen = agen; idx_remove = i
                else:
                    if dist < best_metric: best_metric = dist; target_agen = agen; idx_remove = i
            except: continue
    
    if target_agen:
        # EKSEKUSI PENGANTARAN
        path_nodes_truk.append(target_agen['Node'])
        current_node = target_agen['Node']
        current_capacity -= target_agen['Demand']
        unvisited_agen.pop(idx_remove)
        
        # Catat Agen ini masuk ke Trip yang sedang berjalan
        current_trip_agents.append({
            'name': target_agen['Nama'],
            'lat': target_agen['lat'], 'lon': target_agen['lon'],
            'order': visit_order_total,
            'demand': target_agen['Demand']
        })
        visit_order_total += 1
        print(f"   Drop {target_agen['Demand']} paket @ {target_agen['Nama']} (Sisa Cap: {current_capacity})")
        
    else:
        # KAPASITAS HABIS / TIDAK CUKUP -> BALIK GUDANG
        print("   Balik Warehouse (REFILL) -> Trip Selesai")
        path_nodes_truk.append(node_warehouse)
        
        # 1. Simpan Data Trip Ini
        visited_agents_per_trip.append(current_trip_agents)
        
        # 2. Reset untuk Trip Baru
        current_node = node_warehouse
        current_capacity = KAPASITAS_TRUK + 15
        current_trip_agents = [] # Kosongkan pencatat agen
        
        # (Note: path_nodes_truk tetap nyambung terus buat geometri, nanti dipotong di bawah)

# Finalisasi Trip Terakhir
if path_nodes_truk[-1] != node_warehouse:
    path_nodes_truk.append(node_warehouse)
if current_trip_agents:
    visited_agents_per_trip.append(current_trip_agents)

# --- GENERATE GEOMETRI & PISAH PER TRIP ---
# Kita potong path_nodes_truk yang panjang itu menjadi segmen-segmen per trip
temp_segment = []
truk_trips = []

for k in range(len(path_nodes_truk) - 1):
    u = path_nodes_truk[k]; v = path_nodes_truk[k+1]
    try:
        seg = nx.shortest_path(G, u, v, weight='length')
        if k > 0 and len(temp_segment) > 0: seg = seg[1:] # Hindari duplikat node
        temp_segment.extend(seg)
        
        # Jika node tujuan adalah Warehouse, berarti Trip Selesai
        if v == node_warehouse:
            truk_trips.append(temp_segment) # Simpan Trip
            temp_segment = []               # Reset
    except: pass


# --- B. RUTE KURIR (TSP) + CATAT JARAK ---
print("\nMenghitung Rute Kurir (TSP)...")
rute_kurir_final = {}
paket_orders = {} 

grouped = df_paket.groupby('Agen Induk')
for i, (nama, group) in enumerate(grouped):
    node_agen = ox.nearest_nodes(G, group.iloc[0]['Agen Lon'], group.iloc[0]['Agen Lat'])
    unvisited = [{'node': ox.nearest_nodes(G, r['lon'], r['lat']), 'id': r['ID Paket'], 'lat': r['lat'], 'lon': r['lon']} for _, r in group.iterrows()]

    curr = node_agen; path_seq = [node_agen]
    agent_pkt_orders = []; order = 1
    
    while unvisited:
        nearest = None; min_d = float('inf'); idx_rem = -1
        for idx, pkt in enumerate(unvisited):
            try:
                d = nx.shortest_path_length(G, curr, pkt['node'], weight='length')
                if d < min_d: min_d = d; nearest = pkt; idx_rem = idx
            except: continue
        if nearest:
            path_seq.append(nearest['node']); curr = nearest['node']
            unvisited.pop(idx_rem)
            agent_pkt_orders.append({'id': nearest['id'], 'order': order, 'lat': nearest['lat'], 'lon': nearest['lon'], 'jarak_meter': min_d})
            order += 1
        else: break
    path_seq.append(node_agen)
    paket_orders[nama] = agent_pkt_orders
    
    full_path = []
    for k in range(len(path_seq)-1):
        try:
            seg = nx.shortest_path(G, path_seq[k], path_seq[k+1], weight='length')
            if k > 0: seg = seg[1:]
            full_path.extend(seg)
        except: pass
    rute_kurir_final[nama] = full_path

print("Cell 4: Selesai (Multi-Trip Logic).")


[Cell 4: Menghitung Rute 1 Truk (Multi-Trip CVRP)...]
Menghitung Trip Truk (Bolak-balik Warehouse)...
   Drop 8 paket @ JNE Agen Bersaudara (Sisa Cap: 192)
   Drop 8 paket @ Agen JNE Kumbolo (Sisa Cap: 184)
   Drop 8 paket @ JNE 048 Papahan (Sisa Cap: 176)
   Drop 8 paket @ Cabang JNE Karanganyar (Sisa Cap: 168)
   Drop 8 paket @ JNE Mayor Ahmadi (Sisa Cap: 160)
   Drop 8 paket @ JNE Agen 075 (Sisa Cap: 152)
   Drop 9 paket @ JNE Mojolaban (Sisa Cap: 143)
   Drop 8 paket @ JNE Agen 23 Pasar Kliwon (Sisa Cap: 135)
   Drop 9 paket @ JNE Pasar Klewer (Sisa Cap: 126)
   Drop 9 paket @ JNE Agen Gading (Sisa Cap: 117)
   Drop 8 paket @ JNE Agen 064 (Sisa Cap: 109)
   Drop 9 paket @ JNE Citra Damai (Sisa Cap: 100)
   Drop 9 paket @ JNE Gubug Mas 016 (Sisa Cap: 91)
   Drop 9 paket @ JNE Express Slamet Riyadi (Sisa Cap: 82)
   Drop 9 paket @ JNE Express 031 (Sisa Cap: 73)
   Drop 9 paket @ JNE Cantika (Sisa Cap: 64)
   Drop 9 paket @ JNE Gloria (Sisa Cap: 55)
   Drop 9 paket @ JNE Akasia (Sisa

In [27]:
# ==========================================
# 5. VISUALISASI FOLIUM
# ==========================================
from folium.features import DivIcon

print("\n[Cell 5: Visualisasi Layer per Trip...]")
m = folium.Map(location=[WAREHOUSE_LAT, WAREHOUSE_LON], zoom_start=11, tiles='OpenStreetMap')

# Marker Warehouse (Permanen)
folium.Marker(
    [WAREHOUSE_LAT, WAREHOUSE_LON],
    popup="<b>WAREHOUSE PUSAT</b>",
    icon=folium.Icon(color='black', icon='plane', prefix='fa')
).add_to(m)

# Warna Trip (Selang-seling Hijau/Merah)
trip_colors = ['green', 'red']

# --- LOOP SETIAP TRIP UNTUK BIKIN LAYER TERPISAH ---
for i, trip_nodes in enumerate(truk_trips):
    # Nama Layer: "Truk Trip #1", "Truk Trip #2", dst.
    layer_name = f"🚛 Truk Trip #{i+1}"
    layer_trip = folium.FeatureGroup(name=layer_name, show=True)
    
    warna = trip_colors[i % 2]
    
    # 1. Gambar Garis Rute Trip Ini
    if trip_nodes:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in trip_nodes]
        folium.PolyLine(
            coords, 
            color=warna, 
            weight=5, 
            opacity=0.8,
            tooltip=layer_name
        ).add_to(layer_trip)
    
    # 2. Gambar Marker Agen yg Dikunjungi di Trip Ini
    if i < len(visited_agents_per_trip):
        for agent_info in visited_agents_per_trip[i]:
            # Ikon Angka
            icon_html = f"""
                <div style="
                    font-size: 12pt; color: white; background-color: {warna};
                    border: 2px solid white; border-radius: 50%;
                    text-align: center; line-height: 30px; 
                    width: 30px; height: 30px;
                    box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
                    font-weight: bold;">
                    {agent_info['order']}
                </div>
            """
            
            folium.Marker(
                location=[agent_info['lat'], agent_info['lon']],
                popup=f"<b>{agent_info['name']}</b><br>Trip: #{i+1}<br>Urutan Global: {agent_info['order']}",
                tooltip=f"Urutan {agent_info['order']} ({agent_info['name']})",
                icon=DivIcon(icon_size=(30,30), icon_anchor=(15,15), html=icon_html)
            ).add_to(layer_trip)
            
    # Masukkan Layer Trip ini ke Peta
    layer_trip.add_to(m)


# --- LAYER KURIR (Tetap) ---
for nama, nodes in rute_kurir_final.items():
    layer_kurir = folium.FeatureGroup(name=f"Area: {nama}", show=False)
    if nodes:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in nodes]
        folium.PolyLine(coords, color='#32CD32', weight=3, opacity=0.8).add_to(layer_kurir)
        start = nodes[0]
        folium.Marker([G.nodes[start]['y'], G.nodes[start]['x']], popup=f"Base: {nama}", icon=folium.Icon(color='green', icon='motorcycle', prefix='fa')).add_to(layer_kurir)

    if nama in paket_orders:
        for pkg in paket_orders[nama]:
            html_pkg = f'<div style="font-size: 8pt; color: white; background-color: #32CD32; border: 1px solid white; border-radius: 50%; text-align: center; line-height: 18px; width: 18px; height: 18px;">{pkg["order"]}</div>'
            folium.Marker([pkg['lat'], pkg['lon']], popup=pkg['id'], icon=DivIcon(icon_size=(18,18), html=html_pkg)).add_to(layer_kurir)
    layer_kurir.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
print("Cell 5: Peta dengan Checkbox per Trip Siap!")


[Cell 5: Visualisasi Layer per Trip...]
Cell 5: Peta dengan Checkbox per Trip Siap!


In [28]:
# ==========================================
# 6. SIMPAN OUTPUT
# ==========================================
print("\n[Cell 6: Saving Outputs...]")
base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
output_dir = os.path.join(base_dir, nama_folder_output)
if not os.path.exists(output_dir): os.makedirs(output_dir)

# 1. CSV
laporan_final = []

# Mapping: Agen X ada di Trip ke berapa?
agen_di_trip_berapa = {}
for i, agen_list in enumerate(visited_agents_per_trip):
    nama_trip = f"Trip #{i+1}"
    for agen in agen_list:
        agen_di_trip_berapa[agen['name']] = nama_trip

for nama_agen, paket_list in paket_orders.items():
    trip_info = agen_di_trip_berapa.get(nama_agen, "Gagal Angkut")
    
    for pkt in paket_list:
        jarak_km = round(pkt['jarak_meter'] / 1000, 2)
        laporan_final.append({
            'ID Paket': pkt['id'],
            'Agen JNE': nama_agen,
            'Kloter Pengiriman': trip_info, # <--- Labelnya jadi Trip
            'Urutan Kurir': pkt['order'],
            'Jarak Estafet (km)': jarak_km,
            'Lat': pkt['lat'], 'Lon': pkt['lon']
        })

df_laporan = pd.DataFrame(laporan_final)
df_laporan.sort_values(by=['Kloter Pengiriman', 'Agen JNE', 'Urutan Kurir'], inplace=True)

csv_path = os.path.join(output_dir, "laporan_pembagian_area.csv")
df_laporan.to_csv(csv_path, index=False)

# 2. HTML
html_path = os.path.join(output_dir, "peta_distribusi_final.html")
m.save(html_path)

print(f"\n[SELESAI] Project Beres!")
print(f"CSV: {os.path.basename(csv_path)}")
print(f"HTML: {os.path.basename(html_path)}")


[Cell 6: Saving Outputs...]

[SELESAI] Project Beres!
CSV: laporan_pembagian_area.csv
HTML: peta_distribusi_final.html
